In this notebook we will use a <a href="https://scikit-learn.org/stable/" target="_blank" rel="noopener">`scikit-learn`</a> model created earlier to predict the age of an abalone from its physical measurements and sex. You can find more information about the problem domain <a href="https://archive.ics.uci.edu/dataset/1/abalone" target="_blank" rel="noopener">here</a>.

We will be using a generic prediction UDF script. To execute queries and load data from Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

Prior to running this notebook one needs to complete the follow steps:
1. [Create generic scikit-learn prediction UDF script](sklearn_predict_udf.ipynb).
2. [Train a model on the Abalone](sklearn_train_abalone.ipynb).

In [5]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
from dataclasses import dataclass

@dataclass
class SandboxConfig:
    EXTERNAL_HOST_NAME = "192.168.124.93"
    HOST_PORT = "8888"

    @property
    def EXTERNAL_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.HOST_PORT}"""

    USER = "sys"
    PASSWORD = "exasol"
    BUCKETFS_PORT = "6666"
    BUCKETFS_USER = "w"
    BUCKETFS_PASSWORD = "write"
    BUCKETFS_USE_HTTPS = False
    BUCKETFS_SERVICE = "bfsdefault"
    BUCKETFS_BUCKET = "default"

    @property
    def EXTERNAL_BUCKETFS_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.BUCKETFS_PORT}"""

    @property
    def BUCKETFS_URL_PREFIX(self):
        return "https://" if self.BUCKETFS_USE_HTTPS else "http://"

    @property
    def BUCKETFS_PATH(self):
        # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
        return f"/buckets/{self.BUCKETFS_SERVICE}/{self.BUCKETFS_BUCKET}"

    SCRIPT_LANGUAGE_NAME = "PYTHON3_60"
    UDF_FLAVOR = "python3-ds-EXASOL-6.0.0"
    UDF_RELEASE= "20190116"
    UDF_CLIENT = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
    SCHEMA = "IDA"

    @property
    def SCRIPT_LANGUAGES(self):
        return f"""{self.SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}?lang=python#buckets/{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}/exaudf/{self.UDF_CLIENT}""";

    @property
    def connection_params(self):
        return {"dns": self.EXTERNAL_HOST, "user": self.USER, "password": self.PASSWORD, "compression": True}

    @property
    def params(self):
        return {
            "script_languages": self.SCRIPT_LANGUAGES,
            "script_language_name": self.SCRIPT_LANGUAGE_NAME,
            "schema": self.SCHEMA,
            "BUCKETFS_PORT": self.BUCKETFS_PORT,
            "BUCKETFS_USER": self.BUCKETFS_USER,
            "BUCKETFS_PASSWORD": self.BUCKETFS_PASSWORD,
            "BUCKETFS_USE_HTTPS": self.BUCKETFS_USE_HTTPS,
            "BUCKETFS_BUCKET": self.BUCKETFS_BUCKET,
            "BUCKETFS_PATH": self.BUCKETFS_PATH
        }

conf = SandboxConfig()

Let's do predictions on the data we have in the ABALONE_TEST table. This table also includes a column with ground truth labels. We will use it to access the performance of our predictor. In the code below we will add the ROWID to the output columns (as required by the generic prediction UDF). This will allow us to link the result to the ground truth.

In [6]:
import pyexasol
from stopwatch import Stopwatch

LABEL_COLUMN = 'RINGS'
params = dict(conf.params)
params['test_table'] = 'ABALONE_TEST'
params['model_path'] = f'{conf.BUCKETFS_PATH}/abalone_svm_model.pkl'

stopwatch = Stopwatch()

with pyexasol.connect(dsn=conf.EXTERNAL_HOST, user=conf.USER, password=conf.PASSWORD, compression=True) as conn:
    # Get the list of feature columns
    sql = 'SELECT * FROM {schema!i}.{test_table!i} LIMIT 1'
    df_tmp = conn.export_to_pandas(query_or_table=sql, query_params=params)
    params['column_names'] = [f'[{c}]' for c in df_tmp.columns if c != LABEL_COLUMN]

    # Get the predictions for all rows in the TEST table calling the prediction UDF.
    # Provide the model path and the row id int the first two parameters.
    sql = f'SELECT {{schema!i}}.SKLEARN_PREDICT({{model_path!s}}, ROWID, {{column_names!r}}) ' \
        f'emits ([sample_id] DECIMAL(20,0), [{LABEL_COLUMN}] DOUBLE) FROM {{schema!i}}.{{test_table!i}}'
    df_pred = conn.export_to_pandas(query_or_table=sql, query_params=params)

print(f"Getting predictions took: {stopwatch}")
df_pred.head()

Getting predictions took: 3.02s


,sample_id,RINGS
0,18446744073709551679,6.755714
1,18446744073709551850,6.925311
2,18446744073709552278,6.716555
3,18446744073709551975,7.657736
4,18446744073709551690,7.762828


We are going to check the performance of our predictor by linking the results to the ground truth labels and computing some regression metrics. This should give us similar results to what we have seen in the [training notebook](sklearn_train_abalone.ipynb).

In [7]:
import pandas as pd
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Get the ground truth labels for the test set.
with pyexasol.connect(dsn=conf.EXTERNAL_HOST, user=conf.USER, password=conf.PASSWORD, compression=True) as conn:
    sql = f'SELECT ROWID AS [sample_id], [{LABEL_COLUMN}] FROM {{schema!i}}.{{test_table!i}}'
    df_true = conn.export_to_pandas(query_or_table=sql, query_params=params)

# Merge predictions and the ground truth on the sample id.
df_res = pd.merge(left=df_true, right=df_pred, on='sample_id', suffixes=['_true', '_pred'])

print('Mean absolute error:', mean_absolute_error(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))
print('Mean squared error:', mean_squared_error(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))
print('Explained variance:', explained_variance_score(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))

Mean absolute error: 1.4693975242754505
Mean squared error: 4.734913393345643
Explained variance: 0.5572854305515393
